In [1]:
import numpy as np
import pandas as pd

from scipy.stats import skew, kurtosis

import os
import gc

#pd.set_option('max_columns', None)

In [2]:
PATH_TO_DATA = ('D:/Py/DataFrames/Santander_Value_Prediction_Challenge(KAGGLE)/')

In [3]:
def del_id_and_target(X):
    if 'target' in X.columns:
        data = X.drop(['ID', 'target'], axis=1)
    else:
        data = X.drop(['ID'], axis=1)
    return data

In [4]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'input/train.csv'))
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'input/test.csv'))

## Info

In [5]:
train_df.head(3)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [6]:
test_df.head(3)

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)

Train shape: (4459, 4993)
Test shape: (49342, 4992)


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 4992 entries, ID to 9fc776466
dtypes: float64(4991), object(1)
memory usage: 1.8+ GB


## Checking missing values

In [10]:
constans_columns = train_df.loc[:, (train_df == train_df.iloc[0]).all()].columns.tolist()
print(len(constans_columns))

256


In [11]:
def find_too_freq_values(threshold=0.98, constant_value=0):
    cols_with_too_freq_values = []
    for column in train_df.columns:
        counts = train_df[column].value_counts()
        try:
            counts[constant_value]
        except KeyError:
            continue
        value_fraction = counts[constant_value] / len(train_df)
        if value_fraction >= threshold:
            cols_with_too_freq_values.append(column)
            
    return cols_with_too_freq_values

In [12]:
%%time
cols_with_too_freq_values = find_too_freq_values()
print(len(cols_with_too_freq_values))

2870
Wall time: 6.72 s


In [13]:
cols_to_remove = list(set(constans_columns) | set(cols_with_too_freq_values))
print(len(cols_to_remove))

2870


In [14]:
cols_to_use = test_df.columns.tolist()[1:] # Remove "ID"
cols_to_use = [x for x in cols_to_use if x not in cols_to_remove]

In [15]:
train_df = train_df[cols_to_use + ['ID', 'target']]
test_df = test_df[cols_to_use + ['ID']]

In [16]:
print('Threshold 0.98 freq. values')
print('Train:', train_df.shape)
print('Test: ', test_df.shape)

Threshold 0.98 freq. values
Train: (4459, 2123)
Test:  (49342, 2122)


In [17]:
train_df.to_csv(os.path.join(PATH_TO_DATA, 'input/train_threshold_098.csv'), index=False)
test_df.to_csv(os.path.join(PATH_TO_DATA, 'input/test_threshold_098.csv'), index=False)

## Aggregations

In [5]:
# READ
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'input/train_threshold_098.csv'))
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'input/test_threshold_098.csv'))

In [14]:
def aggregate_row(row):
    non_zero_values = row.iloc[row.nonzero()]
    if len(non_zero_values.value_counts())>1:
        aggs = {'non_zero_min': non_zero_values.min(),
                'non_zero_max': non_zero_values.max(),
                'non_zero_mean': non_zero_values.mean(),
                'non_zero_median': non_zero_values.median(),
                'non_zero_sum': non_zero_values.sum(),
                'non_zero_std': non_zero_values.std(),
                'non_zero_skewness': skew(non_zero_values),
                'non_zero_kurtosis': kurtosis(non_zero_values),
                'non_zero_q1': np.percentile(non_zero_values, q=25),
                'non_zero_q3': np.percentile(non_zero_values, q=75),
                'non_zero_log_min': np.log1p(non_zero_values.astype('float64')).min(),
                'non_zero_log_max': np.log1p(non_zero_values.astype('float64')).max(),
                'non_zero_log_mean': np.log1p(non_zero_values.astype('float64')).mean(),
                'non_zero_log_median': np.log1p(non_zero_values.astype('float64')).median(),
                'non_zero_log_sum': np.log1p(non_zero_values.astype('float64')).sum(),
                'non_zero_log_std': np.log1p(non_zero_values.astype('float64')).std(),
                'non_zero_log_skewness': skew(np.log1p(non_zero_values.astype('float64'))),
                'non_zero_log_kurtosis':  kurtosis(np.log1p(non_zero_values.astype('float64'))) ,
                'non_zero_log_q1': np.percentile(np.log1p(non_zero_values.astype('float64')), q=25),
                'non_zero_log_q3': np.percentile(np.log1p(non_zero_values.astype('float64')), q=75),
                'non_zero_count': non_zero_values.count(),
                'non_zero_fraction': non_zero_values.count() / row.count()
                }
    else: 
        aggs = {'non_zero_min': np.nan,
                'non_zero_max': np.nan,
                'non_zero_mean': np.nan,
                'non_zero_median': np.nan,
                'non_zero_sum': np.nan,
                'non_zero_std': np.nan, 
                'non_zero_skewness': np.nan,
                'non_zero_kurtosis': np.nan,
                'non_zero_q1': np.nan, 
                'non_zero_q3': np.nan,
                'non_zero_log_min': np.nan,
                'non_zero_log_max': np.nan, 
                'non_zero_log_mean': np.nan, 
                'non_zero_log_median': np.nan,
                'non_zero_log_sum': np.nan,
                'non_zero_log_std': np.nan, 
                'non_zero_log_skewness': np.nan,
                'non_zero_log_kurtosis': np.nan, 
                'non_zero_log_q1': np.nan,
                'non_zero_log_q3': np.nan,
                'non_zero_count': 0,
                'non_zero_fraction': 0
                }
            
    return pd.Series(aggs)

In [16]:
%%time
train_df_with_row_statistic = del_id_and_target(train_df).apply(aggregate_row, axis=1)
test_df_with_row_statistic = del_id_and_target(test_df).apply(aggregate_row, axis=1)

Wall time: 6min 53s


In [17]:
def dummies_missing(X, missing_value=0):
    missing_mask = np.where(X.values == missing_value, True, False)
    missing_columns = [f'{col}_is_missing' for col in X.columns]
    X_is_missing = pd.DataFrame(missing_mask.astype(int), columns=missing_columns)
    X = X.join(X_is_missing)

    return X

In [18]:
%%time
train_df_binarized = dummies_missing(del_id_and_target(train_df))
test_df_binarized = dummies_missing(del_id_and_target(test_df))

Wall time: 3.03 s


In [19]:
# JOINING Datasets
train_df_with_row_statistic = train_df_with_row_statistic.join(train_df_binarized)
test_df_with_row_statistic = test_df_with_row_statistic.join(test_df_binarized)

In [23]:
train_df_with_row_statistic.to_csv(os.path.join(PATH_TO_DATA, 'train_with_row_statistic_and_bin_thresh098.csv'), index=False)
test_df_with_row_statistic.to_csv(os.path.join(PATH_TO_DATA, 'test_with_row_statistic_and_bin_thresh098.csv'), index=False)

## Create space reduction features

In [18]:
from sklearn.decomposition import PCA, TruncatedSVD, FastICA, FactorAnalysis
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection

In [19]:
feat_extractors_dict = {
    'pca': PCA(n_components=500, random_state=44),
    'tsvd': TruncatedSVD(n_components=50, n_iter=10, random_state=44),
    'fa': FactorAnalysis(n_components=50, random_state=44),
    'srp': SparseRandomProjection(n_components=50, random_state=44),
    'grp': GaussianRandomProjection(n_components=50, eps=0.1, random_state=44)
}

In [20]:
def create_dim_reduction_feats(df, train=False):
    full_X_space_reduction = []
    all_cols = []
    
    for k, v in feat_extractors_dict.items():
        
        if train:
            print(f'Process train {k}')
            X_space_reduction = v.fit_transform(df)
        else:
            print(f'Process {k}')
            X_space_reduction = v.transform(df)
            
        n_components = X_space_reduction.shape[1]
        cols = [str(k) + f'{i}' for i in range(n_components)]
        all_cols += cols
            
        if len(full_X_space_reduction) == 0:
            full_X_space_reduction = X_space_reduction
        else:
            full_X_space_reduction = np.hstack((full_X_space_reduction, X_space_reduction))
    
    new_df = pd.DataFrame(full_X_space_reduction, columns=all_cols, index=df.index)           
    return new_df

In [21]:
%%time
train_space_reduction = create_dim_reduction_feats(del_id_and_target(train_df), train=True)
test_space_reduction = create_dim_reduction_feats(del_id_and_target(test_df))

Process train pca
Process train tsvd
Process train fa
Process train srp
Process train grp
Process pca
Process tsvd
Process fa
Process srp
Process grp
Wall time: 19.7 s


In [22]:
train_space_reduction.head(3)

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,...,grp40,grp41,grp42,grp43,grp44,grp45,grp46,grp47,grp48,grp49
0,-1.508603e+07,-573018.614867,57685.746183,-650521.789298,-421801.577139,-357791.234921,491722.914574,700494.728980,504996.197311,-4.016240e+06,...,-2.460360e+07,-2.010908e+06,2.796613e+07,1.743904e+06,-1.091064e+07,2.242675e+06,-2.475686e+07,4.685232e+07,-1.139506e+07,6.888125e+06
1,-2.131931e+07,-730451.349052,42760.689456,-710564.446199,-469393.988729,-347590.673528,412821.639203,762472.641524,499498.747433,-3.775476e+06,...,1.486469e+06,1.324593e+07,1.263404e+07,7.633258e+06,5.895837e+05,-2.929417e+06,1.565505e+07,3.879774e+06,-7.191871e+05,-2.446653e+06
2,-2.891396e+07,-786310.308309,10106.571984,-809295.515844,-468074.317391,-355196.826444,491272.337027,748926.912399,503339.329014,-3.878444e+06,...,-8.219437e+04,-2.596314e+06,-5.038003e+05,-5.997508e+06,-1.918620e+06,-3.372705e+06,-1.555013e+05,1.573579e+05,-2.900414e+06,-8.553912e+05


In [23]:
test_space_reduction.head(3)

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,...,grp40,grp41,grp42,grp43,grp44,grp45,grp46,grp47,grp48,grp49
0,-2.037303e+07,-8.790100e+05,-357441.425457,-5.315787e+05,791866.457126,-3.515507e+05,95661.115141,523455.910350,446759.612164,-3.382357e+06,...,2.221898e+08,-1.124679e+08,-7.181361e+07,-1.585694e+08,1.052468e+08,-4.459739e+07,2.782583e+08,-1.244453e+08,3.098919e+07,8.840486e+07
1,-3.084624e+07,-7.739477e+05,3205.690104,-7.957675e+05,-471353.891777,-3.340123e+05,447083.962712,761726.456722,503105.547690,-3.822216e+06,...,2.706779e+06,4.108811e+05,-4.746959e+06,3.119373e+06,-7.279593e+06,1.191656e+06,-3.602817e+06,6.099906e+06,2.594159e+06,-3.508032e+06
2,-2.330157e+07,-1.096363e+06,-21789.269278,-1.411854e+06,-939358.460252,2.307687e+06,214006.579221,244931.112407,258829.644561,-3.112630e+06,...,-1.592457e+07,-1.687704e+07,2.441389e+07,-3.193522e+06,1.910976e+06,9.466727e+06,1.082744e+07,2.659513e+06,-1.493686e+06,1.591741e+07


In [24]:
train_space_reduction.to_csv(os.path.join(PATH_TO_DATA, 'train_space_reduction_50comp.csv'), index=False)
test_space_reduction.to_csv(os.path.join(PATH_TO_DATA, 'test_space_reduction_50comp.csv'), index=False)